In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import warnings
warnings.filterwarnings('ignore')

# Set visual style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Libraries loaded successfully")
<jupyter_output>
<empty_output>
<jupyter_text>
1. Load Processed Data from Task 2
<jupyter_code>
# Load the processed data from Task 2
import sys
sys.path.append('../src')

try:
    # Try to load from multiple possible locations
    file_paths = [
        '../data/processed/processed_reviews.csv',
        '../data/processed/processed_reviews_with_sentiment.csv',
        'data/processed/processed_reviews.csv'
    ]
    
    df = None
    for path in file_paths:
        try:
            df = pd.read_csv(path)
            print(f"✅ Data loaded from: {path}")
            print(f"   Shape: {df.shape}")
            break
        except:
            continue
    
    if df is None:
        # Create sample data if file doesn't exist
        print("⚠️ No data file found. Creating sample data for demonstration.")
        np.random.seed(42)
        n_samples = 1200
        
        sample_data = {
            'bank_name': np.random.choice(['CBE', 'BOA', 'Dashen'], n_samples),
            'review_text': [f"Sample review {i}" for i in range(n_samples)],
            'rating': np.random.choice([1, 2, 3, 4, 5], n_samples, p=[0.1, 0.15, 0.25, 0.3, 0.2]),
            'review_date': pd.date_range('2023-01-01', periods=n_samples).date,
            'sentiment_label': np.random.choice(['positive', 'neutral', 'negative'], n_samples, p=[0.4, 0.3, 0.3]),
            'sentiment_score': np.random.uniform(-1, 1, n_samples),
            'cleaned_text': [f"Cleaned review {i}" for i in range(n_samples)]
        }
        df = pd.DataFrame(sample_data)
        
except Exception as e:
    print(f"❌ Error loading data: {e}")
    raise

# Display basic info
print("\n📊 DATA OVERVIEW:")
print("="*50)
print(f"Total reviews: {len(df):,}")
print(f"Columns: {list(df.columns)}")
print(f"\nFirst 5 rows:")
display(df.head())

print("\n📈 BASIC STATISTICS:")
print("="*50)
print(df.describe(include='all'))
<jupyter_output>
<empty_output>
<jupyter_text>
2. Data Quality Assessment
<jupyter_code>
# Check for missing values
print("🔍 MISSING VALUES CHECK:")
print("="*50)
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_data,
    'Percentage': missing_percent
})
display(missing_df[missing_df['Missing Count'] > 0])

# Check data types
print("\n📋 DATA TYPES:")
print("="*50)
print(df.dtypes)

# Check unique values for categorical columns
print("\n🎯 UNIQUE VALUES:")
print("="*50)
categorical_cols = ['bank_name', 'sentiment_label']
for col in categorical_cols:
    if col in df.columns:
        print(f"\n{col}:")
        print(f"  Unique values: {df[col].nunique()}")
        print(f"  Value counts:")
        print(df[col].value_counts())
<jupyter_output>
<empty_output>
<jupyter_text>
3. Distribution Analysis
<jupyter_code>
# Create figure for distributions
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Data Distribution Analysis', fontsize=16, fontweight='bold')

# 1. Rating distribution
ax1 = axes[0, 0]
if 'rating' in df.columns:
    rating_counts = df['rating'].value_counts().sort_index()
    bars = ax1.bar(rating_counts.index.astype(str), rating_counts.values)
    ax1.set_title('Rating Distribution (1-5)', fontweight='bold')
    ax1.set_xlabel('Rating')
    ax1.set_ylabel('Count')
    
    # Add counts on bars
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 3,
                f'{int(height)}', ha='center', va='bottom')

# 2. Sentiment distribution
ax2 = axes[0, 1]
if 'sentiment_label' in df.columns:
    sentiment_counts = df['sentiment_label'].value_counts()
    colors = ['#4CAF50', '#FFC107', '#F44336']  # Green, Yellow, Red
    ax2.pie(sentiment_counts.values, labels=sentiment_counts.index,
            autopct='%1.1f%%', colors=colors, startangle=90)
    ax2.set_title('Sentiment Distribution', fontweight='bold')

# 3. Bank distribution
ax3 = axes[1, 0]
if 'bank_name' in df.columns:
    bank_counts = df['bank_name'].value_counts()
    bars = ax3.bar(bank_counts.index, bank_counts.values)
    ax3.set_title('Reviews per Bank', fontweight='bold')
    ax3.set_xlabel('Bank')
    ax3.set_ylabel('Number of Reviews')
    ax3.tick_params(axis='x', rotation=45)
    
    # Add counts on bars
    for bar in bars:
        height = bar.get_height()
        ax3.text(bar.get_x() + bar.get_width()/2., height + 3,
                f'{int(height)}', ha='center', va='bottom')

# 4. Sentiment score distribution
ax4 = axes[1, 1]
if 'sentiment_score' in df.columns:
    ax4.hist(df['sentiment_score'], bins=30, edgecolor='black', alpha=0.7)
    ax4.set_title('Sentiment Score Distribution', fontweight='bold')
    ax4.set_xlabel('Sentiment Score')
    ax4.set_ylabel('Frequency')
    ax4.axvline(x=0, color='red', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()
<jupyter_output>
<empty_output>
<jupyter_text>
4. Bank Comparison Analysis
<jupyter_code>
print("🏦 BANK COMPARISON ANALYSIS")
print("="*60)

if 'bank_name' in df.columns:
    # Create comparison metrics
    comparison_metrics = []
    
    for bank in df['bank_name'].unique():
        bank_data = df[df['bank_name'] == bank]
        
        metrics = {
            'Bank': bank,
            'Total Reviews': len(bank_data),
            'Avg Rating': bank_data['rating'].mean() if 'rating' in bank_data.columns else None,
            'Avg Sentiment': bank_data['sentiment_score'].mean() if 'sentiment_score' in bank_data.columns else None,
            'Positive %': (bank_data['sentiment_label'] == 'positive').mean() * 100 if 'sentiment_label' in bank_data.columns else None,
            'Negative %': (bank_data['sentiment_label'] == 'negative').mean() * 100 if 'sentiment_label' in bank_data.columns else None,
        }
        comparison_metrics.append(metrics)
    
    # Create comparison DataFrame
    comparison_df = pd.DataFrame(comparison_metrics)
    
    # Display comparison table
    print("\n📊 Performance Metrics by Bank:")
    display(comparison_df.style.background_gradient(subset=['Avg Rating', 'Avg Sentiment', 'Positive %'], cmap='RdYlGn'))
    
    # Create visualization
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle('Bank Performance Comparison', fontsize=14, fontweight='bold')
    
    # 1. Average Rating
    ax1 = axes[0]
    bars1 = ax1.bar(comparison_df['Bank'], comparison_df['Avg Rating'])
    ax1.set_title('Average Rating', fontweight='bold')
    ax1.set_ylabel('Rating (1-5)')
    ax1.set_ylim(0, 5.5)
    ax1.tick_params(axis='x', rotation=45)
    
    # Add values on bars
    for bar in bars1:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                f'{height:.2f}', ha='center', va='bottom')
    
    # 2. Positive Sentiment Percentage
    ax2 = axes[1]
    bars2 = ax2.bar(comparison_df['Bank'], comparison_df['Positive %'])
    ax2.set_title('Positive Reviews Percentage', fontweight='bold')
    ax2.set_ylabel('Percentage (%)')
    ax2.set_ylim(0, 100)
    ax2.tick_params(axis='x', rotation=45)
    
    # Add values on bars
    for bar in bars2:
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height + 1,
                f'{height:.1f}%', ha='center', va='bottom')
    
    # 3. Total Reviews
    ax3 = axes[2]
    bars3 = ax3.bar(comparison_df['Bank'], comparison_df['Total Reviews'])
    ax3.set_title('Total Number of Reviews', fontweight='bold')
    ax3.set_ylabel('Count')
    ax3.tick_params(axis='x', rotation=45)
    
    # Add values on bars
    for bar in bars3:
        height = bar.get_height()
        ax3.text(bar.get_x() + bar.get_width()/2., height + 5,
                f'{int(height)}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    # Statistical comparison
    print("\n📈 STATISTICAL COMPARISON:")
    print("-" * 40)
    
    if 'rating' in df.columns:
        from scipy import stats
        
        banks = df['bank_name'].unique()
        for i in range(len(banks)):
            for j in range(i+1, len(banks)):
                bank1_data = df[df['bank_name'] == banks[i]]['rating'].dropna()
                bank2_data = df[df['bank_name'] == banks[j]]['rating'].dropna()
                
                if len(bank1_data) > 10 and len(bank2_data) > 10:
                    t_stat, p_value = stats.ttest_ind(bank1_data, bank2_data, equal_var=False)
                    print(f"{banks[i]} vs {banks[j]}:")
                    print(f"  t-statistic: {t_stat:.3f}, p-value: {p_value:.4f}")
                    if p_value < 0.05:
                        print(f"  ⚠️  Significant difference in ratings (p < 0.05)")
                    else:
                        print(f"  ✅ No significant difference in ratings")
                    print()
<jupyter_output>
<empty_output>
<jupyter_text>
5. Text Analysis & Word Clouds
<jupyter_code>
# Text analysis for each bank
print("📝 TEXT ANALYSIS BY BANK")
print("="*60)

if 'review_text' in df.columns and 'bank_name' in df.columns:
    for bank in df['bank_name'].unique():
        print(f"\n🏦 {bank}:")
        print("-" * 30)
        
        bank_reviews = df[df['bank_name'] == bank]['review_text'].dropna()
        
        if len(bank_reviews) > 0:
            # Calculate basic text statistics
            review_lengths = bank_reviews.str.len()
            word_counts = bank_reviews.str.split().str.len()
            
            print(f"Total reviews: {len(bank_reviews)}")
            print(f"Average characters per review: {review_lengths.mean():.0f}")
            print(f"Average words per review: {word_counts.mean():.0f}")
            print(f"Shortest review: {review_lengths.min()} chars")
            print(f"Longest review: {review_lengths.max()} chars")
            
            # Show sample reviews
            print(f"\nSample Reviews:")
            for i, review in enumerate(bank_reviews.head(3)):
                print(f"  {i+1}. {review[:150]}..." if len(review) > 150 else f"  {i+1}. {review}")

# Generate word clouds
print("\n☁️ WORD CLOUD GENERATION")
print("="*60)

if 'review_text' in df.columns and 'bank_name' in df.columns:
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    fig.suptitle('Word Clouds by Bank', fontsize=16, fontweight='bold')
    
    for idx, bank in enumerate(df['bank_name'].unique()):
        ax = axes[idx]
        bank_reviews = df[df['bank_name'] == bank]['review_text'].dropna()
        
        if len(bank_reviews) > 0:
            # Combine all reviews
            text = ' '.join(bank_reviews.astype(str))
            
            # Generate word cloud
            wordcloud = WordCloud(
                width=400,
                height=300,
                background_color='white',
                max_words=100,
                contour_width=2,
                contour_color='steelblue'
            ).generate(text)
            
            ax.imshow(wordcloud, interpolation='bilinear')
            ax.set_title(f'{bank}\n({len(bank_reviews)} reviews)', fontweight='bold')
            ax.axis('off')
        else:
            ax.text(0.5, 0.5, 'No reviews available', 
                   ha='center', va='center', fontsize=12)
            ax.axis('off')
    
    plt.tight_layout()
    plt.show()
<jupyter_output>
<empty_output>
<jupyter_text>
6. Sentiment Analysis Deep Dive
<jupyter_code>
print("😊 SENTIMENT ANALYSIS DEEP DIVE")
print("="*60)

if 'sentiment_label' in df.columns and 'rating' in df.columns:
    # Create cross-tabulation of sentiment vs rating
    sentiment_rating = pd.crosstab(df['sentiment_label'], df['rating'])
    
    print("\n📊 Sentiment vs Rating Cross-tabulation:")
    display(sentiment_rating.style.background_gradient(cmap='Blues'))
    
    # Create visualization
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # 1. Heatmap of sentiment vs rating
    ax1 = axes[0]
    sns.heatmap(sentiment_rating, annot=True, fmt='d', cmap='YlOrRd', ax=ax1)
    ax1.set_title('Sentiment vs Rating Distribution', fontweight='bold')
    ax1.set_xlabel('Rating')
    ax1.set_ylabel('Sentiment')
    
    # 2. Sentiment by bank
    ax2 = axes[1]
    if 'bank_name' in df.columns:
        sentiment_by_bank = pd.crosstab(df['bank_name'], df['sentiment_label'], normalize='index') * 100
        sentiment_by_bank.plot(kind='bar', stacked=True, ax=ax2, 
                              color=['#4CAF50', '#FFC107', '#F44336'])
        ax2.set_title('Sentiment Distribution by Bank', fontweight='bold')
        ax2.set_xlabel('Bank')
        ax2.set_ylabel('Percentage (%)')
        ax2.legend(title='Sentiment')
        ax2.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Analyze sentiment-score relationship
    if 'sentiment_score' in df.columns:
        print("\n📈 Sentiment Score Analysis:")
        print("-" * 40)
        
        # Correlation between rating and sentiment score
        correlation = df['rating'].corr(df['sentiment_score'])
        print(f"Correlation between rating and sentiment score: {correlation:.3f}")
        
        if correlation > 0.5:
            print("✅ Strong positive correlation: Higher ratings correspond to more positive sentiment")
        elif correlation > 0.3:
            print("⚠️ Moderate positive correlation")
        elif correlation > 0:
            print("⚠️ Weak positive correlation")
        else:
            print("❌ No or negative correlation - check sentiment analysis model")
<jupyter_output>
<empty_output>
<jupyter_text>
7. Time Series Analysis (if dates available)
<jupyter_code>
print("📅 TIME SERIES ANALYSIS")
print("="*60)

if 'review_date' in df.columns:
    # Convert to datetime
    df['review_date'] = pd.to_datetime(df['review_date'], errors='coerce')
    
    # Filter out invalid dates
    date_df = df.dropna(subset=['review_date'])
    
    if len(date_df) > 0:
        # Resample by month
        date_df.set_index('review_date', inplace=True)
        
        # Create monthly aggregates
        monthly_stats = date_df.resample('M').agg({
            'rating': 'mean',
            'sentiment_score': 'mean'
        })
        
        # Count reviews per month
        monthly_counts = date_df.resample('M').size()
        
        # Create time series plot
        fig, axes = plt.subplots(2, 1, figsize=(14, 10))
        
        # 1. Monthly review counts
        ax1 = axes[0]
        monthly_counts.plot(ax=ax1, marker='o', linewidth=2)
        ax1.set_title('Monthly Review Volume', fontweight='bold')
        ax1.set_xlabel('Month')
        ax1.set_ylabel('Number of Reviews')
        ax1.grid(True, alpha=0.3)
        
        # Add trend line
        try:
            from scipy import stats
            x = np.arange(len(monthly_counts))
            y = monthly_counts.values
            slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
            trend_line = slope * x + intercept
            ax1.plot(monthly_counts.index, trend_line, 'r--', alpha=0.7, 
                    label=f'Trend (slope: {slope:.2f})')
            ax1.legend()
        except:
            pass
        
        # 2. Monthly average rating and sentiment
        ax2 = axes[1]
        ax2.plot(monthly_stats.index, monthly_stats['rating'], 
                marker='o', linewidth=2, label='Average Rating')
        ax2.set_xlabel('Month')
        ax2.set_ylabel('Rating (1-5)', color='blue')
        ax2.tick_params(axis='y', labelcolor='blue')
        ax2.grid(True, alpha=0.3)
        
        # Add sentiment on secondary axis
        ax2_sentiment = ax2.twinx()
        ax2_sentiment.plot(monthly_stats.index, monthly_stats['sentiment_score'], 
                          marker='s', linewidth=2, color='red', label='Average Sentiment')
        ax2_sentiment.set_ylabel('Sentiment Score', color='red')
        ax2_sentiment.tick_params(axis='y', labelcolor='red')
        
        # Combine legends
        lines1, labels1 = ax2.get_legend_handles_labels()
        lines2, labels2 = ax2_sentiment.get_legend_handles_labels()
        ax2.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
        
        ax2.set_title('Monthly Average Rating and Sentiment', fontweight='bold')
        
        plt.tight_layout()
        plt.show()
        
        # Print time-based insights
        print("\n⏰ TIME-BASED INSIGHTS:")
        print("-" * 40)
        print(f"Date range: {date_df.index.min().date()} to {date_df.index.max().date()}")
        print(f"Total months of data: {len(monthly_counts)}")
        
        # Busiest month
        busiest_month = monthly_counts.idxmax()
        print(f"Busiest month: {busiest_month.strftime('%B %Y')} ({monthly_counts.max()} reviews)")
        
        # Best rated month
        if 'rating' in monthly_stats.columns:
            best_month = monthly_stats['rating'].idxmax()
            print(f"Best rated month: {best_month.strftime('%B %Y')} (Rating: {monthly_stats['rating'].max():.2f})")
    else:
        print("⚠️ No valid date data available for time series analysis")
else:
    print("⚠️ 'review_date' column not found in dataset")
<jupyter_output>
<empty_output>
<jupyter_text>
8. Preliminary Insights for Task 4
<jupyter_code>
print("🔍 PRELIMINARY INSIGHTS FOR TASK 4")
print("="*60)

# Initialize insights dictionary
insights = {
    'drivers': {},
    'pain_points': {},
    'recommendations': {}
}

# Analyze each bank separately
if 'bank_name' in df.columns:
    for bank in df['bank_name'].unique():
        print(f"\n🏦 ANALYSIS FOR {bank}:")
        print("-" * 40)
        
        bank_data = df[df['bank_name'] == bank]
        
        # Drivers (from positive reviews)
        positive_reviews = bank_data[bank_data['sentiment_label'] == 'positive']
        if len(positive_reviews) > 0:
            print(f"\n✅ POTENTIAL SATISFACTION DRIVERS ({len(positive_reviews)} positive reviews):")
            
            # Analyze common themes in positive reviews (simplified)
            common_words = {}
            if 'review_text' in positive_reviews.columns:
                # Simple word frequency analysis
                all_text = ' '.join(positive_reviews['review_text'].astype(str).str.lower())
                words = all_text.split()
                from collections import Counter
                word_counts = Counter(words)
                
                # Filter out common stopwords
                stopwords = {'the', 'and', 'to', 'of', 'a', 'in', 'is', 'it', 'that', 'for', 'on', 'with', 'as', 'this', 'by', 'i', 'not'}
                meaningful_words = {word: count for word, count in word_counts.items() 
                                   if word not in stopwords and len(word) > 3}
                
                top_words = sorted(meaningful_words.items(), key=lambda x: x[1], reverse=True)[:5]
                
                print("  Top words in positive reviews:")
                for word, count in top_words:
                    print(f"    • '{word}': {count} mentions")
                
                insights['drivers'][bank] = [word for word, _ in top_words[:3]]
        
        # Pain points (from negative reviews)
        negative_reviews = bank_data[bank_data['sentiment_label'] == 'negative']
        if len(negative_reviews) > 0:
            print(f"\n❌ POTENTIAL PAIN POINTS ({len(negative_reviews)} negative reviews):")
            
            if 'review_text' in negative_reviews.columns:
                # Simple word frequency analysis
                all_text = ' '.join(negative_reviews['review_text'].astype(str).str.lower())
                words = all_text.split()
                from collections import Counter
                word_counts = Counter(words)
                
                # Filter out common stopwords
                stopwords = {'the', 'and', 'to', 'of', 'a', 'in', 'is', 'it', 'that', 'for', 'on', 'with', 'as', 'this', 'by', 'i', 'not'}
                meaningful_words = {word: count for word, count in word_counts.items() 
                                   if word not in stopwords and len(word) > 3}
                
                top_words = sorted(meaningful_words.items(), key=lambda x: x[1], reverse=True)[:5]
                
                print("  Top words in negative reviews:")
                for word, count in top_words:
                    print(f"    • '{word}': {count} mentions")
                
                insights['pain_points'][bank] = [word for word, _ in top_words[:3]]
        
        # Preliminary recommendations
        print(f"\n💡 PRELIMINARY RECOMMENDATIONS:")
        
        # Based on rating analysis
        if 'rating' in bank_data.columns:
            avg_rating = bank_data['rating'].mean()
            if avg_rating < 3.0:
                print("  ⚠️ Urgent improvement needed - average rating below 3.0")
                print("    • Conduct user interviews to understand core issues")
                print("    • Prioritize bug fixes and stability improvements")
            elif avg_rating < 4.0:
                print("  ⚠️ Room for improvement - average rating between 3.0-4.0")
                print("    • Address top pain points from negative reviews")
                print("    • Enhance existing features based on positive feedback")
            else:
                print("  ✅ Good performance - average rating above 4.0")
                print("    • Maintain quality and address minor issues")
                print("    • Consider adding new features requested by users")
        
        # Based on review volume
        total_reviews = len(bank_data)
        if total_reviews < 300:
            print(f"  📊 Low review volume ({total_reviews} reviews)")
            print("    • Encourage more users to leave reviews")
            print("    • Implement in-app feedback prompts")
        
        print()

# Summary of insights
print("\n" + "="*60)
print("📋 SUMMARY OF PRELIMINARY INSIGHTS")
print("="*60)

for bank in df['bank_name'].unique() if 'bank_name' in df.columns else []:
    print(f"\n{bank}:")
    if bank in insights['drivers']:
        print(f"  Drivers: {', '.join(insights['drivers'][bank])}")
    if bank in insights['pain_points']:
        print(f"  Pain Points: {', '.join(insights['pain_points'][bank])}")

print("\n" + "="*60)
print("✅ EXPLORATORY ANALYSIS COMPLETED")
print("="*60)
<jupyter_output>
<empty_output>
<jupyter_text>
9. Save Analysis Results
<jupyter_code>
# Save processed insights for Task 4
import json
import os

# Create output directory
output_dir = '../reports/exploratory'
os.makedirs(output_dir, exist_ok=True)

# Save insights
insights_file = os.path.join(output_dir, 'preliminary_insights.json')
with open(insights_file, 'w') as f:
    json.dump(insights, f, indent=2)

print(f"💾 Preliminary insights saved to: {insights_file}")

# Save summary statistics
summary_stats = {
    'total_reviews': len(df),
    'banks_analyzed': list(df['bank_name'].unique()) if 'bank_name' in df.columns else [],
    'date_range': {
        'start': str(df['review_date'].min()) if 'review_date' in df.columns else None,
        'end': str(df['review_date'].max()) if 'review_date' in df.columns else None
    } if 'review_date' in df.columns else None,
    'average_rating': float(df['rating'].mean()) if 'rating' in df.columns else None,
    'sentiment_distribution': dict(df['sentiment_label'].value_counts()) if 'sentiment_label' in df.columns else None
}

stats_file = os.path.join(output_dir, 'summary_statistics.json')
with open(stats_file, 'w') as f:
    json.dump(summary_stats, f, indent=2)

print(f"📊 Summary statistics saved to: {stats_file}")

# Save key visualizations
visualization_dir = os.path.join(output_dir, 'visualizations')
os.makedirs(visualization_dir, exist_ok=True)

# Save the main comparison plot
if 'bank_name' in df.columns:
    fig, ax = plt.subplots(figsize=(10, 6))
    comparison_df[['Bank', 'Avg Rating', 'Positive %']].set_index('Bank').plot(kind='bar', ax=ax)
    ax.set_title('Bank Performance Comparison', fontweight='bold')
    ax.set_ylabel('Score')
    ax.tick_params(axis='x', rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(visualization_dir, 'bank_comparison.png'), dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"📈 Visualization saved to: {visualization_dir}/bank_comparison.png")

print("\n" + "="*60)
print("🎉 EXPLORATORY ANALYSIS NOTEBOOK COMPLETED")
print("="*60)
